In [1]:
import os
from pathlib import Path

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["TF_FORCE_UNIFIED_MEMORY"] = "1"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.4"

DATA_DIR = Path.cwd().parent / "data/rte"

In [2]:
import numpy as np
import jax
import jax.numpy as jnp
import tensorflow as tf
from deeprte.dataset import (
    _load_and_split_dataset,
    Split,
    _shard,
    load,
    convert_dataset,
)


In [3]:
ds = load(
    Split.TRAIN_AND_VALID,
    is_training=True,
    batch_dims=[5, 10, 500],
    data_dir=DATA_DIR / "rte_2d_converted.npz",
)

ds.element_spec


Processing data, shapes are: {'sigma_t': (500, 40, 40), 'sigma_a': (500, 40, 40), 'psi_bc': (500, 40, 12), 'psi_label': (500, 40, 40, 24), 'phi': (500, 40, 40)}
Processing grid, shapes are: {'w_angle': (24,), 'v': (24, 2), 'r': (40, 40, 2), 'rv_prime': (40, 12, 4), 'w_prime': (40, 12)}


In [10]:
from deeprte.dataset import get_nest_dict_shape

for d in ds.take(1):
    print(get_nest_dict_shape(d))
    r, v = d["interior"][0], d["interior"][1]
    print(np.sum((r[1] - [0]) ** 2))
    print(np.sum((v[1] - v[0]) ** 2))

{'interior': (TensorShape([5, 500, 2]), TensorShape([5, 500, 2]), F(x=TensorShape([5, 1600, 2]), y=TensorShape([5, 10, 1600, 2])), F(x=TensorShape([5, 480, 4]), y=TensorShape([5, 10, 480]))), 'label': TensorShape([5, 10, 500])}
321.7138
0.0
